In [1]:
import pandas as pd
import numpy as np

## 处理订单信息

In [2]:
#已使用excel从WBL手动筛选出 非合约、去钢、SOC=0
data_order = pd.read_excel("/Users/lilfish/Desktop/实验室/中远/data/TSN_NSH/1TSN_NSH_WBL.xlsx", sheet_name='Sheet2')
data_voyage = pd.read_excel("/Users/lilfish/Desktop/实验室/中远/data/TSN_NSH/TSN_NSH_CNTR_LEG.xlsx", sheet_name='Sheet1')

In [3]:
info_order_voyage = pd.merge(data_voyage, data_order, on="WBL_NUM")
info_order_voyage.drop(columns = ['POL_FCIL_NME', 'POD_FCIL_NME', 'WBL_SOURCE', 'CGO_TYPE_NME', 'CGO_SUB_TYPE_NME', 'BKG_MTHD', 'SOC', 'TRANS_TYPE', 'F_POL_CDE', 'F_POL_NME', 'L_POD_CDE', 'L_POD_NME' ], inplace = True)

In [4]:
info_order_voyage.head()

WBL_NUM                     WBL_CNTR_UUID  SEQ_NUM POL_CDE POL_NME  \
0  Q017717971  3204c39f33a84d0c91e224a833a1cc38        1     HHA      黄骅   
1  Q017717971  3204c39f33a84d0c91e224a833a1cc38        2     TSN      新港   
2  Q017717971  3204c39f33a84d0c91e224a833a1cc38        3     NSH      南沙   
3  Q017717971  c4a55b4860704c1db4cbbcc1a1f38a16        1     HHA      黄骅   
4  Q017717971  c4a55b4860704c1db4cbbcc1a1f38a16        2     TSN      新港   

  POL_FCIL_CDE POD_CDE POD_NME POD_FCIL_CDE       MAIN_SVVD         WBL_AUD_DT  
0        HHA01     TSN      新港        TSN05  IC10-QB3-061 S  2019/12/6 8:39:58  
1        TSN05     NSH      南沙        NSH03  IC10-QB3-061 S  2019/12/6 8:39:58  
2        NSH03     HAK      海口        HAK02  IC10-QB3-061 S  2019/12/6 8:39:58  
3        HHA01     TSN      新港        TSN05  IC10-QB3-061 S  2019/12/6 8:39:58  
4        TSN05     NSH      南沙        NSH03  IC10-QB3-061 S  2019/12/6 8:39:58

In [5]:
# Zhida = info_order_voyage[info_order_voyage['SEQ_NUM'] == 1]
# Zhida.head()
Zhida = info_order_voyage.groupby("WBL_CNTR_UUID", as_index=False).size() # 根据箱子ID聚类
Zhida = Zhida[Zhida["size"] == 1] #挑选出size = 1的箱子（其实就是直达的箱子，如果是中转的箱子会有多条信息）
Zhida

WBL_CNTR_UUID  size
7       000861028da2492089b20b55151676b6     1
13      000c7dc92fb64457817aa6e15e0ba3ec     1
22      001196329e124b82b01d63d6bee50dd4     1
25      0014627c2e264bdcb15815aeccbad6a0     1
35      001b5d57c36f4b22a89593a625edaa74     1
...                                  ...   ...
103448  ffe5f67c32d14184bef13936fea79300     1
103460  ffe9c7146c544f3cac815691a445d675     1
103478  fff1294963d64e8da8ce998d82f3f2f7     1
103491  fff931788cdd47f89ddea84b6364fa40     1
103505  ffffc74c2938434dbf9289780bc8b341     1

[10927 rows x 2 columns]

In [6]:
Zhida = pd.merge(Zhida, info_order_voyage, on="WBL_CNTR_UUID")
Zhida.drop(columns = ['size'], inplace = True)

In [7]:
Zhida.head()

WBL_CNTR_UUID     WBL_NUM  SEQ_NUM POL_CDE POL_NME  \
0  000861028da2492089b20b55151676b6  Q035162640        1     TSN      新港   
1  000c7dc92fb64457817aa6e15e0ba3ec  Q023487490        1     TSN      新港   
2  001196329e124b82b01d63d6bee50dd4  Q037649570        1     TSN      新港   
3  0014627c2e264bdcb15815aeccbad6a0  Q021221530        1     TSN      新港   
4  001b5d57c36f4b22a89593a625edaa74  Q022964070        1     TSN      新港   

  POL_FCIL_CDE POD_CDE POD_NME POD_FCIL_CDE       MAIN_SVVD  \
0        TSN08     NSH      南沙        NSH03  IC10-QSH-143 S   
1        TSN08     NSH      南沙        NSH03  IC10-R3N-093 S   
2        TSN05     NSH      南沙        NSH03  IC10-NLY-029 S   
3        TSN08     NSH      南沙        NSH03  IC10-R3N-089 S   
4        TSN08     NSH      南沙        NSH03  IC10-R3N-092 S   

           WBL_AUD_DT  
0   2021/2/25 8:58:16  
1  2020/5/25 15:29:57  
2   2021/4/24 9:57:52  
3  2020/3/31 15:22:03  
4  2020/5/13 13:30:20

In [8]:
Zhida.shape[0]

10927

Zhida包含所有直达的**箱子**信息

## Fee

In [9]:
#已筛选过的charge（BUC、OFT）
charge = pd.read_excel("/Users/lilfish/Desktop/实验室/中远/data/TSN_NSH/1TSN_NSH_CNTR_FRT.xlsx", sheet_name='Sheet2')

In [10]:
combined_fee = charge.groupby("WBL_CNTR_UUID").sum("TTL_AMT")

In [11]:
all_info_zhida_fee = pd.merge(Zhida, combined_fee, on="WBL_CNTR_UUID")

In [12]:
all_info_zhida_fee.head()

WBL_CNTR_UUID     WBL_NUM  SEQ_NUM POL_CDE POL_NME  \
0  000861028da2492089b20b55151676b6  Q035162640        1     TSN      新港   
1  0014627c2e264bdcb15815aeccbad6a0  Q021221530        1     TSN      新港   
2  001b5d57c36f4b22a89593a625edaa74  Q022964070        1     TSN      新港   
3  001f2a7f574245d5bd1edf7ec09dc444  Q022648860        1     TSN      新港   
4  001fb4b9ddc24e379876a86679ad1165  Q018555680        1     TSN      新港   

  POL_FCIL_CDE POD_CDE POD_NME POD_FCIL_CDE       MAIN_SVVD  \
0        TSN08     NSH      南沙        NSH03  IC10-QSH-143 S   
1        TSN08     NSH      南沙        NSH03  IC10-R3N-089 S   
2        TSN08     NSH      南沙        NSH03  IC10-R3N-092 S   
3        TSN05     NSH      南沙        NSH03  IC10-Q40-117 S   
4        TSN08     NSH      南沙        NSH03  IC10-QNV-157 S   

           WBL_AUD_DT  TTL_AMT  
0   2021/2/25 8:58:16   2339.0  
1  2020/3/31 15:22:03   2006.0  
2  2020/5/13 13:30:20   1307.0  
3   2020/5/6 12:36:10   1521.0  
4  2019/12/27 9:35:40   2631.0

In [13]:
all_info_zhida_fee.shape[0]

8462

all_info_zhida_fee 包含所有直达的箱子信息以及价格

## 开船时间

In [14]:
all_info_zhida_fee["date"] = all_info_zhida_fee["WBL_AUD_DT"].apply(lambda x: x.split(" ")[0])
all_info_zhida_fee["harbor_pair"] = all_info_zhida_fee["POL_NME"] + ',' + all_info_zhida_fee["POD_NME"]
harbor_pair = list(all_info_zhida_fee["harbor_pair"])
harbor_pair_unique = np.unique(harbor_pair)
all_info_zhida_fee = all_info_zhida_fee[all_info_zhida_fee['harbor_pair'] == '新港,南沙']
all_info_zhida_fee.head()

WBL_CNTR_UUID     WBL_NUM  SEQ_NUM POL_CDE POL_NME  \
0  000861028da2492089b20b55151676b6  Q035162640        1     TSN      新港   
1  0014627c2e264bdcb15815aeccbad6a0  Q021221530        1     TSN      新港   
2  001b5d57c36f4b22a89593a625edaa74  Q022964070        1     TSN      新港   
3  001f2a7f574245d5bd1edf7ec09dc444  Q022648860        1     TSN      新港   
4  001fb4b9ddc24e379876a86679ad1165  Q018555680        1     TSN      新港   

  POL_FCIL_CDE POD_CDE POD_NME POD_FCIL_CDE       MAIN_SVVD  \
0        TSN08     NSH      南沙        NSH03  IC10-QSH-143 S   
1        TSN08     NSH      南沙        NSH03  IC10-R3N-089 S   
2        TSN08     NSH      南沙        NSH03  IC10-R3N-092 S   
3        TSN05     NSH      南沙        NSH03  IC10-Q40-117 S   
4        TSN08     NSH      南沙        NSH03  IC10-QNV-157 S   

           WBL_AUD_DT  TTL_AMT        date harbor_pair  
0   2021/2/25 8:58:16   2339.0   2021/2/25       新港,南沙  
1  2020/3/31 15:22:03   2006.0   2020/3/31       新港,南沙  
2  2020/5/13 13:30:20   1307.0   2020/5/13       新港,南沙  
3   2020/5/6 12:36:10   1521.0    2020/5/6       新港,南沙  
4  2019/12/27 9:35:40   2631.0  2019/12/27       新港,南沙

all_info_zhida_fee 包含所有新港-南沙，直达的箱子信息、价格、售卖日期

In [15]:
SVVD_SAILING_DATE_data = pd.read_excel("/Users/lilfish/Desktop/实验室/中远/data/EXPECTED_END_SALE_DATE_vxlsx.xlsx", sheet_name='Sheet1')

In [16]:
SVVD_SAILING_DATE_data = SVVD_SAILING_DATE_data.rename(columns={"SVVD": "MAIN_SVVD", "FCIL_CDE": "POL_FCIL_CDE"})
SVVD_SAILING_DATE_data.head()

MAIN_SVVD POL_FCIL_CDE    BERTH_DEP_DT_LOC SCH_STATE
0  C1-CKG33QZ-210802 S        CKG33 2021-08-02 00:00:00  longterm
1  C1-CKG33QZ-210802 S        QZH01 2021-08-04 00:00:00  longterm
2  C1-CKG33QZ-210802 S        QZH02 2021-08-04 00:05:00  longterm
3  C2-CKG09QZ-210703 S        CKG09 2021-07-03 00:00:00  longterm
4  C2-CKG09QZ-210703 S        QZH01 2021-07-05 00:00:00  longterm

In [17]:
all_info_saildate = pd.merge(all_info_zhida_fee, SVVD_SAILING_DATE_data, on=['MAIN_SVVD', 'POL_FCIL_CDE'])
all_info_saildate.drop(columns = ['SCH_STATE'], inplace = True)
all_info_saildate.head(8)

WBL_CNTR_UUID     WBL_NUM  SEQ_NUM POL_CDE POL_NME  \
0  000861028da2492089b20b55151676b6  Q035162640        1     TSN      新港   
1  16636f7918374699a4be2f459e02451d  Q035315840        1     TSN      新港   
2  1b1bf439cb4240e3990fe59db7642025  Q035360380        1     TSN      新港   
3  2104c60deccf4b6c812c968d06cc9f65  Q035322180        1     TSN      新港   
4  2d9fb1bcd3d545d4bebcd2ca7760c731  Q035358400        1     TSN      新港   
5  2df56a9a973e43b8928a61ab16d1fa76  Q035358400        1     TSN      新港   
6  3cc26a6f50574019972316cda68e571a  Q035348770        1     TSN      新港   
7  46c87a7c8f3e4c2888f1e71da18eed69  Q035201590        1     TSN      新港   

  POL_FCIL_CDE POD_CDE POD_NME POD_FCIL_CDE       MAIN_SVVD  \
0        TSN08     NSH      南沙        NSH03  IC10-QSH-143 S   
1        TSN08     NSH      南沙        NSH03  IC10-QSH-143 S   
2        TSN08     NSH      南沙        NSH03  IC10-QSH-143 S   
3        TSN08     NSH      南沙        NSH03  IC10-QSH-143 S   
4        TSN08     NSH      南沙        NSH03  IC10-QSH-143 S   
5        TSN08     NSH      南沙        NSH03  IC10-QSH-143 S   
6        TSN08     NSH      南沙        NSH03  IC10-QSH-143 S   
7        TSN08     NSH      南沙        NSH03  IC10-QSH-143 S   

           WBL_AUD_DT  TTL_AMT       date harbor_pair    BERTH_DEP_DT_LOC  
0   2021/2/25 8:58:16   2339.0  2021/2/25       新港,南沙 2021-03-06 23:00:00  
1   2021/3/1 10:56:29   2339.0   2021/3/1       新港,南沙 2021-03-06 23:00:00  
2   2021/3/2 10:15:47   2339.0   2021/3/2       新港,南沙 2021-03-06 23:00:00  
3   2021/3/1 12:29:45   2339.0   2021/3/1       新港,南沙 2021-03-06 23:00:00  
4    2021/3/2 9:55:42   2339.0   2021/3/2       新港,南沙 2021-03-06 23:00:00  
5    2021/3/2 9:55:42   2339.0   2021/3/2       新港,南沙 2021-03-06 23:00:00  
6    2021/3/2 8:23:47   2339.0   2021/3/2       新港,南沙 2021-03-06 23:00:00  
7  2021/2/25 16:52:29   2339.0  2021/2/25       新港,南沙 2021-03-06 23:00:00

In [18]:
all_info_saildate.to_csv("saildate_TSN_NSH.csv", encoding='utf_8_sig')

In [19]:
def output_sample(df):
    output_data = pd.DataFrame({'MAIN_SVVD': 0,
                                    'POL_CDE': 0,
                                    'POL_NME': 0,
                                    'POD_CDE': 0,
                                    'POD_NME': 0,
                                    'WBL_INTEVAL': 0,
                                    'MAIN_SVVD_SAILING_DATE': 0,
                                    'NUM': 0,
                                    'PRICE': 0,
                                    'binom': 0}, index=[1])
    
    SVVD = list(df["MAIN_SVVD"].unique()) #找出所有的SVVD
    for one_SVVD in SVVD:
        df_SVVD = df[df['MAIN_SVVD'] == one_SVVD] #某个航线的数据
        df_SVVD = df_SVVD.sort_values(by='WBL_AUD_DT', ascending=True) #根据日期升序排列
        list_dates = df_SVVD["date"].to_list() #所有的dates
#         intervals = pd.date_range(pd.to_datetime(list_dates[0]), pd.to_datetime(list_dates[-1]), freq='2H') #生成等间隔时间
        int_time_tmp = list_dates[0]
        count_WBL = 0 #订单数
        list_AMT = [] #每个订单的费用
        count_AMT = 0 #总费用
        count_tmp = 0
        fore_index = df_SVVD.index.tolist()[0] 
        for index,row in df_SVVD.iterrows(): 
            int_time = pd.to_datetime(row['WBL_AUD_DT']).floor('2H') #按2小时间隔去整
                    
            if int_time != int_time_tmp: 
                print('改变前:', int_time_tmp)
                print('当前时段：', int_time)
#                 intervals = pd.date_range(pd.to_datetime(int_time_tmp), pd.to_datetime(int_time), freq='2H')

                if int_time_tmp != list_dates[0]:
                    for ii in np.unique(list_AMT):
                        count_tmp = list_AMT.count(ii)
                        count_AMT += count_tmp/count_WBL*ii
                    print('总费用:', count_AMT)
                    data_append = pd.Series({'MAIN_SVVD': df_SVVD.loc[fore_index, 'MAIN_SVVD'],
                                        'POL_CDE': df_SVVD.loc[fore_index, 'POL_CDE'],
                                        'POL_NME': df_SVVD.loc[fore_index, 'POL_NME'],
                                        'POD_CDE': df_SVVD.loc[fore_index, 'POD_CDE'],
                                        'POD_NME': df_SVVD.loc[fore_index, 'POD_NME'],
                                        'WBL_INTEVAL': int_time_tmp,
                                        'MAIN_SVVD_SAILING_DATE': df_SVVD.loc[fore_index, 'BERTH_DEP_DT_LOC'],
                                        'NUM': count_WBL,
                                        'PRICE': count_AMT,
                                        'binom': 1})
                    output_data = output_data.append(data_append, ignore_index=True)

                intervals = pd.date_range(int_time_tmp, int_time, freq='2H', closed='right')
                print(intervals)
                for jj in intervals:
                    data_append = pd.Series({'MAIN_SVVD': df_SVVD.loc[fore_index, 'MAIN_SVVD'],
                                    'POL_CDE': df_SVVD.loc[fore_index, 'POL_CDE'],
                                    'POL_NME': df_SVVD.loc[fore_index, 'POL_NME'],
                                    'POD_CDE': df_SVVD.loc[fore_index, 'POD_CDE'],
                                    'POD_NME': df_SVVD.loc[fore_index, 'POD_NME'],
                                    'WBL_INTEVAL': jj,
                                    'MAIN_SVVD_SAILING_DATE': df_SVVD.loc[fore_index, 'BERTH_DEP_DT_LOC'],
                                    'NUM': 0,
                                    'PRICE': 0,
                                    'binom': 0})
                    output_data = output_data.append(data_append, ignore_index=True)
                int_time_tmp = int_time
                print('改变后:', int_time_tmp)
                
                count_WBL = 1
                list_AMT = [df_SVVD.loc[index, 'TTL_AMT']]
                count_AMT = 0
                count_tmp = 0
                
            elif int_time == int_time_tmp:
                count_WBL += 1
                list_AMT.append(df_SVVD.loc[index, 'TTL_AMT'])
                print('当前时段：', int_time)
            
            fore_index = index
        
        # 一个航线的循环结束
        for ii in np.unique(list_AMT):
                    count_tmp = list_AMT.count(ii)
                    count_AMT += count_tmp/count_WBL*ii
                    print('count_AMT:', count_AMT)
        data_append = pd.Series({'MAIN_SVVD': df_SVVD.loc[fore_index, 'MAIN_SVVD'],
                                        'POL_CDE': df_SVVD.loc[fore_index, 'POL_CDE'],
                                        'POL_NME': df_SVVD.loc[fore_index, 'POL_NME'],
                                        'POD_CDE': df_SVVD.loc[fore_index, 'POD_CDE'],
                                        'POD_NME': df_SVVD.loc[fore_index, 'POD_NME'],
                                        'WBL_INTEVAL': int_time_tmp,
                                        'MAIN_SVVD_SAILING_DATE': df_SVVD.loc[fore_index, 'BERTH_DEP_DT_LOC'],
                                        'NUM': count_WBL,
                                        'PRICE': count_AMT,
                                        'binom': 1})
        output_data = output_data.append(data_append, ignore_index=True)
    return output_data

In [28]:
def output_format(df):
    df.drop([0], axis=0, inplace = True) #删除第一行
    df.drop_duplicates(subset = ['MAIN_SVVD', 'WBL_INTEVAL'], keep = 'last', inplace = True) #删除重复的前一项
    drop_list = ['00:00:00', '02:00:00', '04:00:00', '06:00:00', '20:00:00', '22:00:00']
    for ii in drop_list:
        df.drop(df[df['WBL_INTEVAL'].apply(lambda x: str(x).split(' ')[1]) == ii].index, inplace = True)
    return df

In [25]:
output1 = output_sample(all_info_saildate)

改变前: 2021/2/25
当前时段： 2021-02-25 16:00:00
DatetimeIndex(['2021-02-25 02:00:00', '2021-02-25 04:00:00',
               '2021-02-25 06:00:00', '2021-02-25 08:00:00',
               '2021-02-25 10:00:00', '2021-02-25 12:00:00',
               '2021-02-25 14:00:00', '2021-02-25 16:00:00'],
              dtype='datetime64[ns]', freq='2H')
改变后: 2021-02-25 16:00:00
改变前: 2021-02-25 16:00:00
当前时段： 2021-02-25 08:00:00
总费用: 2339.0
DatetimeIndex([], dtype='datetime64[ns]', freq='2H')
改变后: 2021-02-25 08:00:00
改变前: 2021-02-25 08:00:00
当前时段： 2021-02-27 08:00:00
总费用: 2339.0
DatetimeIndex(['2021-02-25 10:00:00', '2021-02-25 12:00:00',
               '2021-02-25 14:00:00', '2021-02-25 16:00:00',
               '2021-02-25 18:00:00', '2021-02-25 20:00:00',
               '2021-02-25 22:00:00', '2021-02-26 00:00:00',
               '2021-02-26 02:00:00', '2021-02-26 04:00:00',
               '2021-02-26 06:00:00', '2021-02-26 08:00:00',
               '2021-02-26 10:00:00', '2021-02-26 12:00:00',
         

/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:50: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  intervals = pd.date_range(int_time_tmp, int_time, freq='2H', closed='right')
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be rem

改变后: 2021-03-01 10:00:00
改变前: 2021-03-01 10:00:00
当前时段： 2021-03-01 12:00:00
总费用: 2339.0
DatetimeIndex(['2021-03-01 12:00:00'], dtype='datetime64[ns]', freq='2H')
改变后: 2021-03-01 12:00:00
当前时段： 2021-03-01 12:00:00
当前时段： 2021-03-01 12:00:00
改变前: 2021-03-01 12:00:00
当前时段： 2021-03-02 10:00:00
总费用: 2339.0
DatetimeIndex(['2021-03-01 14:00:00', '2021-03-01 16:00:00',
               '2021-03-01 18:00:00', '2021-03-01 20:00:00',
               '2021-03-01 22:00:00', '2021-03-02 00:00:00',
               '2021-03-02 02:00:00', '2021-03-02 04:00:00',
               '2021-03-02 06:00:00', '2021-03-02 08:00:00',
               '2021-03-02 10:00:00'],
              dtype='datetime64[ns]', freq='2H')
改变后: 2021-03-02 10:00:00
当前时段： 2021-03-02 10:00:00
改变前: 2021-03-02 10:00:00
当前时段： 2021-03-02 08:00:00
总费用: 2339.0
DatetimeIndex([], dtype='datetime64[ns]', freq='2H')
改变后: 2021-03-02 08:00:00
当前时段： 2021-03-02 08:00:00
当前时段： 2021-03-02 08:00:00
当前时段： 2021-03-02 08:00:00
当前时段： 2021-03-02 08:00:00
当前时段： 202

/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be rem

...

/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:50: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  intervals = pd.date_range(int_time_tmp, int_time, freq='2H', closed='right')
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:50: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  intervals = pd.date_range(int_time_tmp, int

DatetimeIndex(['2021-04-20 02:00:00', '2021-04-20 04:00:00',
               '2021-04-20 06:00:00', '2021-04-20 08:00:00',
               '2021-04-20 10:00:00', '2021-04-20 12:00:00',
               '2021-04-20 14:00:00', '2021-04-20 16:00:00'],
              dtype='datetime64[ns]', freq='2H')
改变后: 2021-04-20 16:00:00
当前时段： 2021-04-20 16:00:00
当前时段： 2021-04-20 16:00:00
count_AMT: 3139.0
改变前: 2020/7/6
当前时段： 2020-07-06 16:00:00
DatetimeIndex(['2020-07-06 02:00:00', '2020-07-06 04:00:00',
               '2020-07-06 06:00:00', '2020-07-06 08:00:00',
               '2020-07-06 10:00:00', '2020-07-06 12:00:00',
               '2020-07-06 14:00:00', '2020-07-06 16:00:00'],
              dtype='datetime64[ns]', freq='2H')
改变后: 2020-07-06 16:00:00
count_AMT: 2200.0
改变前: 2021/1/21
当前时段： 2021-01-21 14:00:00
DatetimeIndex(['2021-01-21 02:00:00', '2021-01-21 04:00:00',
               '2021-01-21 06:00:00', '2021-01-21 08:00:00',
               '2021-01-21 10:00:00', '2021-01-21 12:00:00',
          

/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:48: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:50: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  intervals = pd.date_range(int_time_tmp, int_time, freq='2H', closed='right')
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:94: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

改变后: 2021-05-05 16:00:00
当前时段： 2021-05-05 16:00:00
count_AMT: 3339.0
改变前: 2020/1/20
当前时段： 2020-01-20 08:00:00
DatetimeIndex(['2020-01-20 02:00:00', '2020-01-20 04:00:00',
               '2020-01-20 06:00:00', '2020-01-20 08:00:00'],
              dtype='datetime64[ns]', freq='2H')
改变后: 2020-01-20 08:00:00
count_AMT: 2295.0
改变前: 2021/2/16
当前时段： 2021-02-16 10:00:00
DatetimeIndex(['2021-02-16 02:00:00', '2021-02-16 04:00:00',
               '2021-02-16 06:00:00', '2021-02-16 08:00:00',
               '2021-02-16 10:00:00'],
              dtype='datetime64[ns]', freq='2H')
改变后: 2021-02-16 10:00:00
当前时段： 2021-02-16 10:00:00
count_AMT: 2339.0
改变前: 2021/3/3
当前时段： 2021-03-03 12:00:00
DatetimeIndex(['2021-03-03 02:00:00', '2021-03-03 04:00:00',
               '2021-03-03 06:00:00', '2021-03-03 08:00:00',
               '2021-03-03 10:00:00', '2021-03-03 12:00:00'],
              dtype='datetime64[ns]', freq='2H')
改变后: 2021-03-03 12:00:00
count_AMT: 2339.0


/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_84616/2303247498.py:63: FutureWarning: The frame.append method is deprecated and will be rem

In [26]:
output1.head(20)

MAIN_SVVD POL_CDE POL_NME POD_CDE POD_NME          WBL_INTEVAL  \
0                0       0       0       0       0                    0   
1   IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 02:00:00   
2   IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 04:00:00   
3   IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 06:00:00   
4   IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 08:00:00   
5   IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 10:00:00   
6   IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 12:00:00   
7   IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 14:00:00   
8   IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 16:00:00   
9   IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 16:00:00   
10  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 08:00:00   
11  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 10:00:00   
12  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 12:00:00   
13  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 14:00:00   
14  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 16:00:00   
15  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 18:00:00   
16  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 20:00:00   
17  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 22:00:00   
18  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-26 00:00:00   
19  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-26 02:00:00   

   MAIN_SVVD_SAILING_DATE  NUM   PRICE  binom  
0                       0    0     0.0      0  
1     2021-03-06 23:00:00    0     0.0      0  
2     2021-03-06 23:00:00    0     0.0      0  
3     2021-03-06 23:00:00    0     0.0      0  
4     2021-03-06 23:00:00    0     0.0      0  
5     2021-03-06 23:00:00    0     0.0      0  
6     2021-03-06 23:00:00    0     0.0      0  
7     2021-03-06 23:00:00    0     0.0      0  
8     2021-03-06 23:00:00    0     0.0      0  
9     2021-03-06 23:00:00    1  2339.0      1  
10    2021-03-06 23:00:00    1  2339.0      1  
11    2021-03-06 23:00:00    0     0.0      0  
12    2021-03-06 23:00:00    0     0.0      0  
13    2021-03-06 23:00:00    0     0.0      0  
14    2021-03-06 23:00:00    0     0.0      0  
15    2021-03-06 23:00:00    0     0.0      0  
16    2021-03-06 23:00:00    0     0.0      0  
17    2021-03-06 23:00:00    0     0.0      0  
18    2021-03-06 23:00:00    0     0.0      0  
19    2021-03-06 23:00:00    0     0.0      0

In [29]:
output_format(output1)

MAIN_SVVD POL_CDE POL_NME POD_CDE POD_NME          WBL_INTEVAL  \
10     IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 08:00:00   
11     IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 10:00:00   
12     IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 12:00:00   
13     IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 14:00:00   
14     IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 16:00:00   
...               ...     ...     ...     ...     ...                  ...   
18683  IC10-QSH-142 S     TSN      新港     NSH      南沙  2021-02-16 08:00:00   
18685  IC10-QSH-142 S     TSN      新港     NSH      南沙  2021-02-16 10:00:00   
18689  IC26-NQY-028 S     TSN      新港     NSH      南沙  2021-03-03 08:00:00   
18690  IC26-NQY-028 S     TSN      新港     NSH      南沙  2021-03-03 10:00:00   
18692  IC26-NQY-028 S     TSN      新港     NSH      南沙  2021-03-03 12:00:00   

      MAIN_SVVD_SAILING_DATE  NUM   PRICE  binom  
10       2021-03-06 23:00:00    1  2339.0      1  
11       2021-03-06 23:00:00    0     0.0      0  
12       2021-03-06 23:00:00    0     0.0      0  
13       2021-03-06 23:00:00    0     0.0      0  
14       2021-03-06 23:00:00    0     0.0      0  
...                      ...  ...     ...    ...  
18683    2021-02-21 23:00:00    0     0.0      0  
18685    2021-02-21 23:00:00    2  2339.0      1  
18689    2021-03-07 23:00:00    0     0.0      0  
18690    2021-03-07 23:00:00    0     0.0      0  
18692    2021-03-07 23:00:00    1  2339.0      1  

[7755 rows x 10 columns]

In [30]:
output1.head(10)

MAIN_SVVD POL_CDE POL_NME POD_CDE POD_NME          WBL_INTEVAL  \
10  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 08:00:00   
11  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 10:00:00   
12  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 12:00:00   
13  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 14:00:00   
14  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 16:00:00   
15  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-25 18:00:00   
22  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-26 08:00:00   
23  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-26 10:00:00   
24  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-26 12:00:00   
25  IC10-QSH-143 S     TSN      新港     NSH      南沙  2021-02-26 14:00:00   

   MAIN_SVVD_SAILING_DATE  NUM   PRICE  binom  
10    2021-03-06 23:00:00    1  2339.0      1  
11    2021-03-06 23:00:00    0     0.0      0  
12    2021-03-06 23:00:00    0     0.0      0  
13    2021-03-06 23:00:00    0     0.0      0  
14    2021-03-06 23:00:00    0     0.0      0  
15    2021-03-06 23:00:00    0     0.0      0  
22    2021-03-06 23:00:00    0     0.0      0  
23    2021-03-06 23:00:00    0     0.0      0  
24    2021-03-06 23:00:00    0     0.0      0  
25    2021-03-06 23:00:00    0     0.0      0

In [31]:
output1.to_csv("data_TSN_NSH.csv", encoding='utf_8_sig')

## 测试

In [171]:
data_test = pd.read_excel("/Users/lilfish/Desktop/test.xlsx", sheet_name='Sheet1')

In [286]:
output_test = output_sample(data_test)
output_test

改变前: 2019-12-10 00:00:00
当前时段： 2019-12-10 12:00:00
DatetimeIndex(['2019-12-10 02:00:00', '2019-12-10 04:00:00',
               '2019-12-10 06:00:00', '2019-12-10 08:00:00',
               '2019-12-10 10:00:00', '2019-12-10 12:00:00'],
              dtype='datetime64[ns]', freq='2H')
改变后: 2019-12-10 12:00:00
当前时段： 2019-12-10 12:00:00
当前时段： 2019-12-10 12:00:00
当前时段： 2019-12-10 12:00:00
当前时段： 2019-12-10 12:00:00
当前时段： 2019-12-10 12:00:00
当前时段： 2019-12-10 12:00:00
当前时段： 2019-12-10 12:00:00
改变前: 2019-12-10 12:00:00
当前时段： 2019-12-12 12:00:00
总费用: 4116.0
DatetimeIndex(['2019-12-10 14:00:00', '2019-12-10 16:00:00',
               '2019-12-10 18:00:00', '2019-12-10 20:00:00',
               '2019-12-10 22:00:00', '2019-12-11 00:00:00',
               '2019-12-11 02:00:00', '2019-12-11 04:00:00',
               '2019-12-11 06:00:00', '2019-12-11 08:00:00',
               '2019-12-11 10:00:00', '2019-12-11 12:00:00',
               '2019-12-11 14:00:00', '2019-12-11 16:00:00',
               '201

/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_41117/1861223268.py:50: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  intervals = pd.date_range(int_time_tmp, int_time, freq='2H', closed='right')
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_41117/1861223268.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_41117/1861223268.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_41117/1861223268.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

改变后: 2020-02-11 08:00:00
改变前: 2020-02-11 08:00:00
当前时段： 2020-02-11 10:00:00
总费用: 2678.0
DatetimeIndex(['2020-02-11 10:00:00'], dtype='datetime64[ns]', freq='2H')
改变后: 2020-02-11 10:00:00
改变前: 2020-02-11 10:00:00
当前时段： 2020-02-11 12:00:00
总费用: 2978.0
DatetimeIndex(['2020-02-11 12:00:00'], dtype='datetime64[ns]', freq='2H')
改变后: 2020-02-11 12:00:00
当前时段： 2020-02-11 12:00:00
当前时段： 2020-02-11 12:00:00
当前时段： 2020-02-11 12:00:00
count_AMT: 2678.0


/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_41117/1861223268.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_41117/1861223268.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_41117/1861223268.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_data = output_data.append(data_append, ignore_index=True)
/var/folders/9f/pnn38y9d3r567_p72yn_q_mm0000gn/T/ipykernel_41117/1861223268.py:63: FutureWarning: The frame.append method is deprecated and will be rem

MAIN_SVVD POL_CDE POL_NME POD_CDE POD_NME          WBL_INTEVAL  \
0                0       0       0       0       0                    0   
1   IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 02:00:00   
2   IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 04:00:00   
3   IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 06:00:00   
4   IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 08:00:00   
5   IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 10:00:00   
6   IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 12:00:00   
7   IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 12:00:00   
8   IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 14:00:00   
9   IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 16:00:00   
10  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 18:00:00   
11  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 20:00:00   
12  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 22:00:00   
13  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 00:00:00   
14  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 02:00:00   
15  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 04:00:00   
16  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 06:00:00   
17  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 08:00:00   
18  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 10:00:00   
19  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 12:00:00   
20  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 14:00:00   
21  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 16:00:00   
22  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 18:00:00   
23  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 20:00:00   
24  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 22:00:00   
25  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-12 00:00:00   
26  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-12 02:00:00   
27  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-12 04:00:00   
28  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-12 06:00:00   
29  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-12 08:00:00   
30  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-12 10:00:00   
31  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-12 12:00:00   
32  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-12 12:00:00   
33  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 02:00:00   
34  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 04:00:00   
35  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 06:00:00   
36  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 08:00:00   
37  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 10:00:00   
38  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 12:00:00   
39  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 12:00:00   
40  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 14:00:00   
41  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 14:00:00   
42  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 16:00:00   
43  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 18:00:00   
44  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 20:00:00   
45  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 22:00:00   
46  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-11 00:00:00   
47  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-11 02:00:00   
48  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-11 04:00:00   
49  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-11 06:00:00   
50  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-11 08:00:00   
51  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-11 08:00:00   
52  IC29-NLY-008 S     YIK      营口

- 最后要删掉binom=1前面一行 （重复）
- 要删除早8:00点前和晚18:00之后的数据

In [287]:
def output_format(df):
    df.drop([0], axis=0, inplace = True) #删除第一行
    df.drop_duplicates('WBL_INTEVAL', keep = 'last', inplace = True) #删除重复的前一项
    drop_list = ['00:00:00', '02:00:00', '04:00:00', '06:00:00', '20:00:00', '22:00:00']
    for ii in drop_list:
        df.drop(output_test[output_test['WBL_INTEVAL'].apply(lambda x: str(x).split(' ')[1]) == ii].index, inplace = True)
    return df

In [288]:
output_test_2 = output_format(output_test)

In [253]:
output_test.drop_duplicates('WBL_INTEVAL', keep = 'last', inplace = True)
output_test.drop([0], axis=0, inplace = True)

In [264]:
drop_list = ['00:00:00', '02:00:00', '04:00:00', '06:00:00', '20:00:00', '22:00:00']
for ii in drop_list:
    output_test.drop(output_test[output_test['WBL_INTEVAL'].apply(lambda x: str(x).split(' ')[1]) == ii].index, inplace = True)
output_test

MAIN_SVVD POL_CDE POL_NME POD_CDE POD_NME          WBL_INTEVAL  \
4   IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 08:00:00   
5   IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 10:00:00   
7   IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 12:00:00   
8   IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 14:00:00   
9   IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 16:00:00   
10  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-10 18:00:00   
17  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 08:00:00   
18  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 10:00:00   
19  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 12:00:00   
20  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 14:00:00   
21  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 16:00:00   
22  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-11 18:00:00   
29  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-12 08:00:00   
30  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-12 10:00:00   
32  IC29-NLY-006 S     YIK      营口     QZH      钦州  2019-12-12 12:00:00   
36  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 08:00:00   
37  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 10:00:00   
39  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 12:00:00   
41  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 14:00:00   
42  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 16:00:00   
43  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-10 18:00:00   
51  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-11 08:00:00   
53  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-11 10:00:00   
55  IC29-NLY-008 S     YIK      营口     QZH      钦州  2020-02-11 12:00:00   

   MAIN_SVVD_SAILING_DATE  NUM   PRICE  binom  
4     2019-12-21 02:00:00    0     0.0      0  
5     2019-12-21 02:00:00    0     0.0      0  
7     2019-12-21 02:00:00    8  4116.0      1  
8     2019-12-21 02:00:00    0     0.0      0  
9     2019-12-21 02:00:00    0     0.0      0  
10    2019-12-21 02:00:00    0     0.0      0  
17    2019-12-21 02:00:00    0     0.0      0  
18    2019-12-21 02:00:00    0     0.0      0  
19    2019-12-21 02:00:00    0     0.0      0  
20    2019-12-21 02:00:00    0     0.0      0  
21    2019-12-21 02:00:00    0     0.0      0  
22    2019-12-21 02:00:00    0     0.0      0  
29    2019-12-21 02:00:00    0     0.0      0  
30    2019-12-21 02:00:00    0     0.0      0  
32    2019-12-21 02:00:00    2  2000.0      1  
36    2020-02-18 22:00:00    0     0.0      0  
37    2020-02-18 22:00:00    0     0.0      0  
39    2020-02-18 22:00:00    1  2678.0      1  
41    2020-02-18 22:00:00    2  2978.0      1  
42    2020-02-18 22:00:00    0     0.0      0  
43    2020-02-18 22:00:00    0     0.0      0  
51    2020-02-18 22:00:00    1  2678.0      1  
53    2020-02-18 22:00:00    1  2978.0      1  
55    2020-02-18 22:00:00    4  2678.0      1

In [254]:
output_test['WBL_INTEVAL'].apply(lambda x: str(x).split(' ')).to_list()

[['2019-12-10', '02:00:00'],
 ['2019-12-10', '04:00:00'],
 ['2019-12-10', '06:00:00'],
 ['2019-12-10', '08:00:00'],
 ['2019-12-10', '10:00:00'],
 ['2019-12-10', '12:00:00'],
 ['2019-12-10', '14:00:00'],
 ['2019-12-10', '16:00:00'],
 ['2019-12-10', '18:00:00'],
 ['2019-12-10', '20:00:00'],
 ['2019-12-10', '22:00:00'],
 ['2019-12-11', '00:00:00'],
 ['2019-12-11', '02:00:00'],
 ['2019-12-11', '04:00:00'],
 ['2019-12-11', '06:00:00'],
 ['2019-12-11', '08:00:00'],
 ['2019-12-11', '10:00:00'],
 ['2019-12-11', '12:00:00'],
 ['2019-12-11', '14:00:00'],
 ['2019-12-11', '16:00:00'],
 ['2019-12-11', '18:00:00'],
 ['2019-12-11', '20:00:00'],
 ['2019-12-11', '22:00:00'],
 ['2019-12-12', '00:00:00'],
 ['2019-12-12', '02:00:00'],
 ['2019-12-12', '04:00:00'],
 ['2019-12-12', '06:00:00'],
 ['2019-12-12', '08:00:00'],
 ['2019-12-12', '10:00:00'],
 ['2019-12-12', '12:00:00'],
 ['2020-02-10', '02:00:00'],
 ['2020-02-10', '04:00:00'],
 ['2020-02-10', '06:00:00'],
 ['2020-02-10', '08:00:00'],
 ['2020-02-10'

In [223]:
def output_format(df):
    for index,row in df.iterrows(): 
        if row['binom'] == 0:
            fore_index = index
            print(fore_index)
        elif row['binom'] == 1:
            df.drop(df.index[fore_index], inplace = True)
            df = df.reset_index(drop=True)
    return df

In [224]:
output_format(output_test)

0
1
2
3
4
5
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
33
34
35
36
37
38
40
42
43
44
45
46
47
48
49
50
52


IndexError: index 52 is out of bounds for axis 0 with size 50

In [111]:
print(pd.date_range(pd.to_datetime('2019/12/31 13:55'), pd.to_datetime('2020/1/1 13:55'), freq='2H') )

DatetimeIndex(['2019-12-31 13:55:00', '2019-12-31 15:55:00',
               '2019-12-31 17:55:00', '2019-12-31 19:55:00',
               '2019-12-31 21:55:00', '2019-12-31 23:55:00',
               '2020-01-01 01:55:00', '2020-01-01 03:55:00',
               '2020-01-01 05:55:00', '2020-01-01 07:55:00',
               '2020-01-01 09:55:00', '2020-01-01 11:55:00',
               '2020-01-01 13:55:00'],
              dtype='datetime64[ns]', freq='2H')


In [85]:
test = [1,2,3,3]
test2 = []
test2 = np.unique(test)
test.count(test2[1])

1

In [86]:
count = 1
count += 2
count

3

In [57]:
test = all_info_saildate.head(10)
for index,row in test.iterrows(): 
    print(row['date'] + ' 08:00:00')

2019/12/10 08:00:00
2019/12/10 08:00:00
2019/12/10 08:00:00
2019/12/10 08:00:00
2019/12/10 08:00:00
2019/12/10 08:00:00
2019/12/10 08:00:00
2019/12/10 08:00:00
2019/12/10 08:00:00
2019/12/10 08:00:00


In [73]:
test['date2'] = pd.Timestamp(test['date'] + ' 08:00:00')

TypeError: Cannot convert input [0    2019/12/10 08:00:00
1    2019/12/10 08:00:00
2    2019/12/10 08:00:00
3    2019/12/10 08:00:00
4    2019/12/10 08:00:00
5    2019/12/10 08:00:00
6    2019/12/10 08:00:00
7    2019/12/10 08:00:00
8    2019/12/10 08:00:00
9    2019/12/10 08:00:00
Name: date, dtype: object] of type <class 'pandas.core.series.Series'> to Timestamp

In [67]:
test

WBL_NUM                     WBL_CNTR_UUID  SEQ_NUM POL_CDE POL_NME  \
0  Q017894430  c4e127ef383644bfbea04994959f4367        1     YIK      营口   
1  Q017894430  fd198a749bdc4c25811908d15a0f3e5a        1     YIK      营口   
2  Q017894430  034a086e55f5480cb9ec579454e37875        1     YIK      营口   
3  Q017894430  3104d259c0144a99be2974ecc5b39776        1     YIK      营口   
4  Q017894470  509f2602931c45b786b7d378a4664147        1     YIK      营口   
5  Q017894470  73fcbefa6e0b454c8c564dd2c6e7be0d        1     YIK      营口   
6  Q017894470  b3fa20c03f5246e4bb57fad248b2ffb3        1     YIK      营口   
7  Q017894470  ce4ed7818d164872bd16b00082181e90        1     YIK      营口   
8  Q017894690  5d3caa586cbf47a797243a7580ac173d        1     YIK      营口   
9  Q017894690  25301aaa6d6249f6af36e9615459b888        1     YIK      营口   

  POL_FCIL_CDE POD_CDE POD_NME POD_FCIL_CDE       MAIN_SVVD  \
0        YIK05     QZH      钦州        QZH02  IC29-NLY-006 S   
1        YIK05     QZH      钦州        QZH02  IC29-NLY-006 S   
2        YIK05     QZH      钦州        QZH02  IC29-NLY-006 S   
3        YIK05     QZH      钦州        QZH02  IC29-NLY-006 S   
4        YIK05     QZH      钦州        QZH02  IC29-NLY-006 S   
5        YIK05     QZH      钦州        QZH02  IC29-NLY-006 S   
6        YIK05     QZH      钦州        QZH02  IC29-NLY-006 S   
7        YIK05     QZH      钦州        QZH02  IC29-NLY-006 S   
8        YIK05     QZH      钦州        QZH02  IC29-NLY-006 S   
9        YIK05     QZH      钦州        QZH02  IC29-NLY-006 S   

            WBL_AUD_DT  TTL_AMT        date harbor_pair    BERTH_DEP_DT_LOC  \
0  2019/12/10 13:36:09     4366  2019/12/10       营口,钦州 2019-12-21 02:00:00   
1  2019/12/10 13:36:09     4366  2019/12/10       营口,钦州 2019-12-21 02:00:00   
2  2019/12/10 13:36:09     4366  2019/12/10       营口,钦州 2019-12-21 02:00:00   
3  2019/12/10 13:36:09     4366  2019/12/10       营口,钦州 2019-12-21 02:00:00   
4  2019/12/10 13:36:09     4366  2019/12/10       营口,钦州 2019-12-21 02:00:00   
5  2019/12/10 13:36:09     4366  2019/12/10       营口,钦州 2019-12-21 02:00:00   
6  2019/12/10 13:36:09     4366  2019/12/10       营口,钦州 2019-12-21 02:00:00   
7  2019/12/10 13:36:09     4366  2019/12/10       营口,钦州 2019-12-21 02:00:00   
8  2019/12/10 13:38:36     4366  2019/12/10       营口,钦州 2019-12-21 02:00:00   
9  2019/12/10 13:38:36     4366  2019/12/10       营口,钦州 2019-12-21 02:00:00   

                 date2  
0  2019/12/10 08:00:00  
1  2019/12/10 08:00:00  
2  2019/12/10 08:00:00  
3  2019/12/10 08:00:00  
4  2019/12/10 08:00:00  
5  2019/12/10 08:00:00  
6  2019/12/10 08:00:00  
7  2019/12/10 08:00:00  
8  2019/12/10 08:00:00  
9  2019/12/10 08:00:00

In [75]:
for index,row in test.iterrows(): 
    print(pd.Timestamp(row['date'] + ' 08:00:00'))

2019-12-10 08:00:00
2019-12-10 08:00:00
2019-12-10 08:00:00
2019-12-10 08:00:00
2019-12-10 08:00:00
2019-12-10 08:00:00
2019-12-10 08:00:00
2019-12-10 08:00:00
2019-12-10 08:00:00
2019-12-10 08:00:00


In [64]:
test['date2'].to_list()

KeyError: 'date2'